In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import sklearn
import lzma
import nltk

from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from scipy.sparse import coo_matrix, hstack
from sklearn.model_selection import cross_validate
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.decomposition import PCA

<h1>Final Notebook</h1>
<p>Final Notebook! We've reached this moment! Normally I'd go all out and work on some really crazy stuff, but other responsibilities have absorbed my time to work on stuff. Nonetheless, I will try out some new things.</p>

<p>